Name : N U Praneeth Reddy

Reg.No :21BAI1500

In [ ]:
!unzip /content/buildings-vs-forests.zip

In [14]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_dir = r"/content/buildings-vs-forests"

classes = {"buildings": 1, "forest": 0}
test_size = 0.3

train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_dir)

    train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)

    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_dir, class_name, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copyfile(src, dst)

    for image in test_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(test_dir, class_name, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copyfile(src, dst)

print("Dataset split completed.")

Dataset split completed.


In [15]:
# Two classes
classes = {"buildings": 1, "forest": 0}

# GLCM

In [16]:
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
import numpy as np
from skimage.io import imread

def extract_glcms(image):
    gray_image = rgb2gray(image)
    gray_image_uint = np.uint8(gray_image * 255)

    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    glcm = graycomatrix(gray_image_uint, distances=distances, angles=angles, symmetric=True, normed=True)

    contrast = graycoprops(glcm, 'contrast').ravel()
    dissimilarity = graycoprops(glcm, 'dissimilarity').ravel()
    homogeneity = graycoprops(glcm, 'homogeneity').ravel()

    return np.concatenate((contrast, dissimilarity, homogeneity))

glcm_features = []
glcm_labels = []

for class_name, label in classes.items():
    class_dir = os.path.join(dataset_dir, "train", class_name)
    images = os.listdir(class_dir)

    for image_name in images:
        image_path = os.path.join(class_dir, image_name)
        image = imread(image_path)
        features = extract_glcms(image)
        glcm_features.append(features)
        glcm_labels.append(label)

# LBP

In [17]:
from skimage.feature import local_binary_pattern

def extract_lbps(image):
    gray_image = rgb2gray(image)

    lbp = local_binary_pattern(gray_image, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    return hist

lbp_features = []
lbp_labels = []

for class_name, label in classes.items():
    class_dir = os.path.join(dataset_dir, "train", class_name)
    images = os.listdir(class_dir)

    for image_name in images:
        image_path = os.path.join(class_dir, image_name)
        image = imread(image_path)
        features = extract_lbps(image)
        lbp_features.append(features)
        lbp_labels.append(label)

#KNN With GLCM

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score

#splitting the data in the ratio 70-30
X_train_glcm, X_test_glcm, y_train_glcm, y_test_glcm = train_test_split(glcm_features, glcm_labels, test_size=0.3, random_state=42)

knn_classifier_glcm = KNeighborsClassifier(n_neighbors=4)
knn_classifier_glcm.fit(X_train_glcm, y_train_glcm)
y_pred_glcm = knn_classifier_glcm.predict(X_test_glcm)


In [19]:
#GLCM Accuracy
accuracy_glcm = accuracy_score(y_test_glcm, y_pred_glcm)
print("kNN with GLCM Features Accuracy:", accuracy_glcm)
#GLCM F1 Score
f1_glcm = f1_score(y_test_glcm, y_pred_glcm)
print("kNN with GLCM Features F1 Score:", f1_glcm)


kNN with GLCM Features Accuracy: 0.8798932384341637
kNN with GLCM Features F1 Score: 0.8710601719197707


# SVC With GLCM

In [20]:
from sklearn.svm import SVC

svm_classifier_glcm = SVC(kernel='linear')
svm_classifier_glcm.fit(X_train_glcm, y_train_glcm)
y_pred_svm_glcm = svm_classifier_glcm.predict(X_test_glcm)

In [21]:
#GLCM Accuracy
accuracy_svm_glcm = accuracy_score(y_test_glcm, y_pred_svm_glcm)
print("SVM with GLCM Features Accuracy:", accuracy_svm_glcm)

#GLCM Accuracy
f1_svm_glcm = f1_score(y_test_glcm, y_pred_glcm)
print("SVM with GLCM Features F1 Score:", f1_svm_glcm)

SVM with GLCM Features Accuracy: 0.9181494661921709
SVM with GLCM Features F1 Score: 0.8710601719197707


# KNN with LBP

In [22]:
from sklearn.neighbors import KNeighborsClassifier

X_train_lbp, X_test_lbp, y_train_lbp, y_test_lbp = train_test_split(lbp_features, lbp_labels, test_size=0.3, random_state=42)
knn_classifier_lbp = KNeighborsClassifier(n_neighbors=4)
knn_classifier_lbp.fit(X_train_lbp, y_train_lbp)
y_pred_lbp = knn_classifier_lbp.predict(X_test_lbp)


In [23]:
#GLCM Accuracy
accuracy_lbp = accuracy_score(y_test_lbp, y_pred_lbp)
print("kNN with LBP Features Accuracy:", accuracy_lbp)

#GLCM Accuracy
f1_lbp = f1_score(y_test_lbp, y_pred_lbp)
print("kNN with LBP Features F1 Score:", f1_lbp)

kNN with LBP Features Accuracy: 0.9395017793594306
kNN with LBP Features F1 Score: 0.9405594405594406


# SVM with LBP

In [24]:
from sklearn.svm import SVC

svm_classifier_lbp = SVC(kernel='linear')
svm_classifier_lbp.fit(X_train_lbp, y_train_lbp)
y_pred_svm_lbp = svm_classifier_lbp.predict(X_test_lbp)

In [25]:
accuracy_svm_lbp = accuracy_score(y_test_lbp, y_pred_svm_lbp)
print("SVM with LBP Features Accuracy:", accuracy_svm_lbp)
f1_svm_lbp = f1_score(y_test_lbp, y_pred_lbp)
print("SVM with LBP Features F1 Score:", f1_svm_glcm)

SVM with LBP Features Accuracy: 0.9359430604982206
SVM with LBP Features F1 Score: 0.8710601719197707
